# Step 1: Inital db setup

In [1]:
# Initial - Setup environment vars before testing anything
import os

os.environ["DEBUG"] = "1"
os.environ["TESTING"] = "0"
# os.environ["DATABASE_URL"] = "sqlite:///:memory:"
# os.environ["TEST_DATABASE_URL"] = "sqlite:///:memory:"

os.environ["DATABASE_URL"] = "sqlite:///test.db"
os.environ["TEST_DATABASE_URL"] = "sqlite:///test.db"

In [2]:
# Initalize database and create schema etc

import logging

from tenacity import after_log, before_log, retry, stop_after_attempt, wait_fixed

from ultron8.api.db.u_sqlite.session import db_session

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

max_tries = 60 * 5  # 5 minutes
wait_seconds = 1

@retry(
    stop=stop_after_attempt(max_tries),
    wait=wait_fixed(wait_seconds),
    before=before_log(logger, logging.INFO),
    after=after_log(logger, logging.WARN),
)
def init():
    try:
        # Try to create session to check if DB is awake
        db_session.execute("SELECT 1")
    except Exception as e:
        logger.error(e)
        raise e


In [3]:
# Get sqlalchemy classes/objects

from ultron8.api.db.u_sqlite.init_db import init_db
from ultron8.api.db.u_sqlite.session import db_session, engine, Session

session = Session()


In [4]:
# make sure all SQL Alchemy models are imported before initializing DB
# otherwise, SQL Alchemy might fail to initialize properly relationships
# for more details: https://github.com/tiangolo/full-stack-fastapi-postgresql/issues/28
from ultron8.api.db.u_sqlite.base import Base

# Tables should be created with Alembic migrations
# But if you don't want to use migrations, create
# the tables un-commenting the next line
Base.metadata.create_all(bind=engine)



2019-07-09 21:04:47,539 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2019-07-09 21:04:47,544 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-07-09 21:04:47,551 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:()


2019-07-09 21:04:47,648 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("guid_tracker")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("guid_tracker")


2019-07-09 21:04:47,665 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [5]:
# Try initializing everything now
logger.info("Initializing service")
init()
logger.info("Service finished initializing")


logger.info("Creating initial data")
init_db(db_session)
logger.info("Initial data created")

INFO:__main__:Initializing service
INFO:__main__:Starting call to '__main__.init', this is the 1st time calling it.


2019-07-09 21:04:47,773 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-07-09 21:04:47,777 INFO sqlalchemy.engine.base.Engine SELECT 1


INFO:sqlalchemy.engine.base.Engine:SELECT 1


...

INFO:sqlalchemy.engine.base.Engine:()
INFO:__main__:Service finished initializing
INFO:__main__:Creating initial data


2019-07-09 21:04:47,807 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


2019-07-09 21:04:47,811 INFO sqlalchemy.engine.base.Engine ('admin', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('admin', 1, 0)
INFO:__main__:Initial data created


# Step 2: Now That we have a working sqlite session, let's try playing w/ our various db_models classes etc

In [6]:
# Step A. Create a pack ( which has actions, triggers, and sensors associated with it )

# Step 3: Import each of the db_models

In [7]:
from ultron8.api.db_models.action import Action
from ultron8.api.db_models.guid import Guid
from ultron8.api.db_models.item import Item
from ultron8.api.db_models.packs import Packs
from ultron8.api.db_models.rule import RuleTypeParameter, RuleType, Rules
from ultron8.api.db_models.sensors import Sensors
# from ultron8.api.db_models.timer import TimeDB
from ultron8.api.db_models.trigger import TriggerType, Trigger, TriggerInstanceDB
from ultron8.api.db_models.user import User

# Step 4: Ok, let us try inserting, and updating data in the database